In [ ]:
!pip install torch torchvision opencv-python matplotlib

In [ ]:
!wget https://www.agentspace.org/download/watch-masks.zip
!unzip watch-masks.zip
!rm watch-masks.zip

In [ ]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as T
from torchvision.utils import draw_segmentation_masks
from torchvision.transforms.v2 import Compose, ToTensor, ConvertImageDtype, Resize
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
import matplotlib.pyplot as plt
import numpy as np
import os, cv2

In [ ]:
class CustomSegmentationDataset(Dataset):
    def __init__(self, root, transforms=None):
        self.root = root
        self.transforms = transforms
        self.imgs = sorted(os.listdir(os.path.join(root, "images")))
        self.masks = sorted(os.listdir(os.path.join(root, "masks")))

    def __getitem__(self, idx):
        img_path = os.path.join(self.root, "images", self.imgs[idx])
        mask_path = os.path.join(self.root, "masks", self.masks[idx])

        img = cv2.imread(img_path)[:, :, ::-1].copy()  # BGR→RGB, add .copy() to fix negative stride issue
        mask = cv2.imread(mask_path, 0)         # grayscale

        # get unique object ids
        obj_ids = np.unique(mask)
        obj_ids = obj_ids[1:]  # remove background id 0

        masks = mask == obj_ids[:, None, None]
        num_objs = len(obj_ids)

        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.ones((num_objs,), dtype=torch.int64)  # all one class
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {
            "boxes": boxes,
            "labels": labels,
            "masks": masks,
            "image_id": image_id,
            "area": area,
            "iscrowd": iscrowd
        }

        if self.transforms is not None:
            img = self.transforms(img)

        return img, target

    def __len__(self):
        return len(self.imgs)

In [ ]:
# Training
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    i = 0
    for images, targets in train_loader:
        images = list(img.cuda() for img in images)
        targets = [{k: v.cuda() for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoch [{epoch}], Step [{i}], Loss: {losses.item():.4f}")
        i += 1

In [ ]:
def get_instance_segmentation_model(num_classes):
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights="DEFAULT")

    # Replace the classifier head
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(
        in_features, num_classes)

    # Replace the mask head
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    model.roi_heads.mask_predictor = MaskRCNNPredictor(
        in_features_mask, hidden_layer, num_classes)

    return model


In [ ]:
# Datasets
transforms = Compose([
    ToTensor(),
    ConvertImageDtype(torch.float32),
    Resize((800,1333)),
])

dataset_train = CustomSegmentationDataset('watch/train', transforms=transforms)
dataset_val = CustomSegmentationDataset('watch/val', transforms=transforms)

train_loader = DataLoader(dataset_train, batch_size=2, shuffle=True, num_workers=2, collate_fn=lambda x: tuple(zip(*x)))
val_loader = DataLoader(dataset_val, batch_size=1, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

In [ ]:
# Model
num_classes = 2  # background + 1 custom class
model = get_instance_segmentation_model(num_classes)
model.cuda()

In [ ]:
# Optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

In [ ]:
# Training
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    i = 0
    for images, targets in train_loader:
        images = list(img.cuda() for img in images)
        targets = [{k: v.cuda() for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoch [{epoch}], Step [{i}], Loss: {losses.item():.4f}")
        i += 1

In [ ]:
# use the trained model
model.eval()
with torch.no_grad():
    img = cv2.imread("dataset/val/images/img001.jpg")[:, :, ::-1]
    img_tensor = T.ToTensor()(img).cuda()
    pred = model([img_tensor])[0]

In [ ]:
masks = pred["masks"] > 0.5
vis = draw_segmentation_masks(torch.from_numpy(img).permute(2,0,1), masks.squeeze(1), alpha=0.5)
plt.imshow(vis.permute(1,2,0))
plt.show()